<a href="https://colab.research.google.com/github/lis-r-barreto/Big-Data-com-PySpark/blob/master/Big_Data_com_PySpark_no_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Big Data com PySpark no Google Colab**

<img class="one" src="https://aprendizadodemaquina.com/media/filer_public_thumbnails/filer_public/3a/65/3a658469-383e-4c1d-b4fb-502229858579/pyspark.png__800x450_q85_crop_subsampling-2.png"  height="150"><br>


A biblioteca PySpark permite você criar seu servidor Apache Spark, trabalhar com grandes volumes de dados e até mesmo fazer streaming em tempo real.

##**1. Configurações**

In [1]:
# Instalando as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

A próxima etapa é configurar as variáveis de ambiente, pois isso habilita o ambiente do Colab a identificar corretamente onde as dependências estão rodando.

Para conseguir “manipular” o terminal e interagir como ele, você pode usar a biblioteca os.

In [2]:
# Configurando as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# Tornando o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

Com tudo pronto, vamos rodar uma sessão local para testar se a instalação funcionou corretamente.

In [23]:
# Iniciando uma sessão local e importando os dados
from pyspark.sql import *

sc = SparkSession.builder.master('local[*]').getOrCreate()

# Download do http para arquivo local
!wget --quiet --show-progress https://raw.githubusercontent.com/lis-r-barreto/Big-Data-com-PySpark/master/data-movielens/movies.csv

!wget --quiet --show-progress https://raw.githubusercontent.com/lis-r-barreto/Big-Data-com-PySpark/master/data-movielens/ratings.csv

# Carregando dados
df_movies = sc.read.csv("./movies.csv", inferSchema=True, header=True)
df_ratings = sc.read.csv("./ratings.csv", inferSchema=True, header=True)

movies.csv.3        100%[===================>] 482.84K  --.-KB/s    in 0.06s   
ratings.csv.3       100%[===================>]   2.37M  12.4MB/s    in 0.2s    


In [14]:
df_movies.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [5]:
df_ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



##**2. Manipulação dos Dados**

Para manipular os dados usaremos os métodos do PySpark, pois são mais rápidos que usar SQL além de ser mais intuitivo e ter menos retrabalho no código. 

In [26]:
inner_join_movies_ratings = df_movies.join(df_ratings, df_movies.movieId == df_ratings.movieId, 'inner')
inner_join_movies_ratings.show(5)

+-------+--------------------+--------------------+------+-------+------+---------+
|movieId|               title|              genres|userId|movieId|rating|timestamp|
+-------+--------------------+--------------------+------+-------+------+---------+
|      1|    Toy Story (1995)|Adventure|Animati...|     1|      1|   4.0|964982703|
|      3|Grumpier Old Men ...|      Comedy|Romance|     1|      3|   4.0|964981247|
|      6|         Heat (1995)|Action|Crime|Thri...|     1|      6|   4.0|964982224|
|     47|Seven (a.k.a. Se7...|    Mystery|Thriller|     1|     47|   5.0|964983815|
|     50|Usual Suspects, T...|Crime|Mystery|Thr...|     1|     50|   5.0|964982931|
+-------+--------------------+--------------------+------+-------+------+---------+
only showing top 5 rows



In [27]:
# Vendo algumas informações sobre os tipos de dados de cada coluna
df_movies_ratings = inner_join_movies_ratings
df_movies_ratings.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [28]:
df_movies_ratings.summary().show()

+-------+----------------+--------------------+------------------+------------------+----------------+------------------+--------------------+
|summary|         movieId|               title|            genres|            userId|         movieId|            rating|           timestamp|
+-------+----------------+--------------------+------------------+------------------+----------------+------------------+--------------------+
|  count|          100836|              100836|            100836|            100836|          100836|            100836|              100836|
|   mean|19435.2957177992|                null|              null|326.12756356856676|19435.2957177992| 3.501556983616962|1.2059460873684695E9|
| stddev|35530.9871987003|                null|              null| 182.6184914635004|35530.9871987003|1.0425292390606342|2.1626103599513078E8|
|    min|               1|"11'09""01 - Sept...|(no genres listed)|                 1|               1|               0.5|           828124615|

In [29]:
df_movies_ratings.drop("userId", "timestamp").show()

+-------+--------------------+--------------------+-------+------+
|movieId|               title|              genres|movieId|rating|
+-------+--------------------+--------------------+-------+------+
|      1|    Toy Story (1995)|Adventure|Animati...|      1|   4.0|
|      3|Grumpier Old Men ...|      Comedy|Romance|      3|   4.0|
|      6|         Heat (1995)|Action|Crime|Thri...|      6|   4.0|
|     47|Seven (a.k.a. Se7...|    Mystery|Thriller|     47|   5.0|
|     50|Usual Suspects, T...|Crime|Mystery|Thr...|     50|   5.0|
|     70|From Dusk Till Da...|Action|Comedy|Hor...|     70|   3.0|
|    101|Bottle Rocket (1996)|Adventure|Comedy|...|    101|   5.0|
|    110|   Braveheart (1995)|    Action|Drama|War|    110|   4.0|
|    151|      Rob Roy (1995)|Action|Drama|Roma...|    151|   5.0|
|    157|Canadian Bacon (1...|          Comedy|War|    157|   5.0|
|    163|    Desperado (1995)|Action|Romance|We...|    163|   5.0|
|    216|Billy Madison (1995)|              Comedy|    216|   

In [30]:
df_movies_ratings.describe("genres").show()

+-------+------------------+
|summary|            genres|
+-------+------------------+
|  count|            100836|
|   mean|              null|
| stddev|              null|
|    min|(no genres listed)|
|    max|           Western|
+-------+------------------+



In [31]:
df_movies_ratings.describe("rating").show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|            100836|
|   mean| 3.501556983616962|
| stddev|1.0425292390606342|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [32]:
df_movies_ratings.select("rating").distinct().show()

+------+
|rating|
+------+
|   3.5|
|   4.5|
|   2.5|
|   1.0|
|   4.0|
|   0.5|
|   3.0|
|   2.0|
|   1.5|
|   5.0|
+------+



In [33]:
df_movies_ratings.select("title", "genres", "rating").filter(col("rating") >= 4.9).show(truncate=False)

+-----------------------------------------+--------------------------------------------+------+
|title                                    |genres                                      |rating|
+-----------------------------------------+--------------------------------------------+------+
|Seven (a.k.a. Se7en) (1995)              |Mystery|Thriller                            |5.0   |
|Usual Suspects, The (1995)               |Crime|Mystery|Thriller                      |5.0   |
|Bottle Rocket (1996)                     |Adventure|Comedy|Crime|Romance              |5.0   |
|Rob Roy (1995)                           |Action|Drama|Romance|War                    |5.0   |
|Canadian Bacon (1995)                    |Comedy|War                                  |5.0   |
|Desperado (1995)                         |Action|Romance|Western                      |5.0   |
|Billy Madison (1995)                     |Comedy                                      |5.0   |
|Dumb & Dumber (Dumb and Dumber) (1994) 

In [34]:
df_movies_ratings\
    .select("title","genres","rating")\
    .orderBy("rating").show(truncate=False)

+---------------------------------------------------+----------------------------------------+------+
|title                                              |genres                                  |rating|
+---------------------------------------------------+----------------------------------------+------+
|As Good as It Gets (1997)                          |Comedy|Drama|Romance                    |0.5   |
|Joy Ride (2001)                                    |Adventure|Thriller                      |0.5   |
|Pearl Harbor (2001)                                |Action|Drama|Romance|War                |0.5   |
|Requiem for a Dream (2000)                         |Drama                                   |0.5   |
|Star Wars: Episode II - Attack of the Clones (2002)|Action|Adventure|Sci-Fi|IMAX            |0.5   |
|Snow Dogs (2002)                                   |Adventure|Children|Comedy               |0.5   |
|Hulk (2003)                                        |Action|Adventure|Sci-Fi      

In [35]:
df_movies_ratings\
    .select("title","genres","rating", round(col("rating"), 1)\
    .alias("rating"))\
    .orderBy("rating", ascending=False).show(truncate=False)

+-------------------------------------------+--------------------------------------------+------+------+
|title                                      |genres                                      |rating|rating|
+-------------------------------------------+--------------------------------------------+------+------+
|Mr. Smith Goes to Washington (1939)        |Drama                                       |5.0   |5.0   |
|Blues Brothers, The (1980)                 |Action|Comedy|Musical                       |5.0   |5.0   |
|Winnie the Pooh and the Blustery Day (1968)|Animation|Children|Musical                  |5.0   |5.0   |
|Seven (a.k.a. Se7en) (1995)                |Mystery|Thriller                            |5.0   |5.0   |
|Three Caballeros, The (1945)               |Animation|Children|Musical                  |5.0   |5.0   |
|Bottle Rocket (1996)                       |Adventure|Comedy|Crime|Romance              |5.0   |5.0   |
|Sword in the Stone, The (1963)             |Animation|

In [36]:
df_movies_ratings\
    .select("title", "genres", "rating")\
    .filter(col("genres")\
    .like("%Fantasy%"))\
    .orderBy("rating", ascending=False)\
    .show(truncate=False)

+-------------------------------------------+---------------------------------------------------------+------+
|title                                      |genres                                                   |rating|
+-------------------------------------------+---------------------------------------------------------+------+
|Black Cauldron, The (1985)                 |Adventure|Animation|Children|Fantasy                     |5.0   |
|Fantasia (1940)                            |Animation|Children|Fantasy|Musical                       |5.0   |
|Conan the Barbarian (1982)                 |Action|Adventure|Fantasy                                 |5.0   |
|Pinocchio (1940)                           |Animation|Children|Fantasy|Musical                       |5.0   |
|Goonies, The (1985)                        |Action|Adventure|Children|Comedy|Fantasy                 |5.0   |
|Wizard of Oz, The (1939)                   |Adventure|Children|Fantasy|Musical                       |5.0   |
|

In [37]:
dimensions = ["title","genres","rating"]
metrics = ["rating"]
dim_df_movies_ratings = df_movies_ratings.groupBy(dimensions).count()
dim_df_movies_ratings.show()

+--------------------+--------------------+------+-----+
|               title|              genres|rating|count|
+--------------------+--------------------+------+-----+
|Starship Troopers...|       Action|Sci-Fi|   3.0|   17|
|    Hangar 18 (1980)|Action|Sci-Fi|Thr...|   5.0|    1|
| Multiplicity (1996)|              Comedy|   2.0|    5|
|Seven Samurai (Sh...|Action|Adventure|...|   5.0|   14|
|A.I. Artificial I...|Adventure|Drama|S...|   4.5|    3|
| Aviator, The (2004)|               Drama|   3.5|    6|
|Planes, Trains & ...|              Comedy|   3.5|    3|
|Flickering Lights...| Action|Comedy|Crime|   4.5|    1|
|    Hitchcock (2012)|               Drama|   3.5|    3|
|       Splash (1984)|Comedy|Fantasy|Ro...|   3.0|   14|
|Miss Congeniality...|        Comedy|Crime|   3.5|   13|
|The Man from U.N....|Action|Adventure|...|   3.5|    5|
|French Connection...|Action|Crime|Thri...|   4.5|    5|
|      Ben-Hur (1959)|Action|Adventure|...|   4.0|   11|
|Hidden Figures (2...|         

In [38]:
groupby_genres = df_movies_ratings.groupBy("genres").count()
groupby_genres.show()

+--------------------+-----+
|              genres|count|
+--------------------+-----+
|Adventure|Sci-Fi|...|   33|
|Comedy|Horror|Thr...|  115|
|Action|Adventure|...|  204|
| Action|Drama|Horror|   20|
|Action|Animation|...|    7|
|Action|Adventure|...|  479|
|    Adventure|Sci-Fi|  128|
|Animation|Childre...|  115|
|Adventure|Childre...|    2|
|Documentary|Music...|    1|
| Adventure|Animation|    2|
| Musical|Romance|War|    9|
|Comedy|Crime|Horr...|    2|
|Adventure|Childre...|   17|
|Crime|Drama|Fanta...|    3|
|Action|Adventure|...|    9|
|   Adventure|Fantasy|  584|
|Comedy|Mystery|Th...|   47|
|Horror|Romance|Sc...|    1|
|Drama|Film-Noir|R...|   33|
+--------------------+-----+
only showing top 20 rows

